# Find a Feasible Mobile Manipulation

1. Offline
    1. Configuration
        1. `M`: X-Y-Cr manipulability map
        2. `Rsize`: the size of the robot-base
    1. Preparation
        1. Convert the `M` to a feasibility map `F` (helical shape).
        2. ~~[SKIP] Convert a cartesian grid `F` to a cylindrical grid `F` by linear interpolation.~~
2. Online
    1. Input
        1. `Pt`: Position of the target object (relative to the robot’s current pose)
        2. `Obs`: Area list of ground obstacles
        3. `Cr`: Constraints on the approach angle (relative to the robot heading)
        4. `Ct`: Constraints on the approach angle (relative to the target heading)
    2. Process
        1. Cut the range of `Cr` from `F` and set it to `Fcut`.
        2. ~~[SKIP] Scan the maximum points for the radius and angle by each circle in `Fcut`.~~
        3. Wipe the `Fcut` in the range of `Ct`.
        4. And extract only the maximum as a `Fmax`.
        5. Remove all obstacle areas from `Fmax` with the offset of `Rsize`.
    3. Output
        1. Candidate poses (sorted in descending order of manipulability)

## 1. Offline

### 1.0. Utils

In [1]:
%matplotlib widget

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
"""
raw:
    [[Cr x y manip],
     [Cr x y manip], ...]
layers:
    [deg]: [[x, ...],
            [y, ...],
            [manip, ...]]
"""

def recarray_to_ndarray(recarray):
    new_shape = recarray.shape + (-1,)  # (225, -1) = (225,) + (-1,)
    ndarray = recarray.view(np.float32).reshape(new_shape)
    return ndarray

def rotation_z(rad):
    c, s = np.cos(rad), np.sin(rad)
    R = np.array((
        (c, -s, 0),
        (s,  c, 0),
        (0,  0, 1)
    ))
    return R

def raw_to_layers(raw):
    def extract_specific_Cr(raw, Cr):
        filter_arr = (raw[:,0] == Cr)
        layer = raw[filter_arr]
        layer = layer[:,1:].copy()
        return np.transpose(layer)
    layers = {}
    for Cr in set(raw[:,0]):
        layers[Cr] = extract_specific_Cr(raw, Cr)
    return layers

def layers_to_raw(layers):
    num_points = 0
    for Cr, layer in layers.items():
        num_points += layer.shape[1]
    raw = np.zeros((num_points, 4), dtype=np.float32)
    idx = 0
    for Cr, layer in layers.items():
        next_idx = idx + layer.shape[1]
        layer_raw = raw[idx:next_idx]
        layer_raw[:,0] += Cr
        layer_raw[:,1:] = np.transpose(layer)
        idx = next_idx
    return raw

def manip_color(manip):
    return (1.0-manip, manip, 0.0, manip)

def scatter_2d(ax, data, xyMinMax, is_feasibility=True, dot_size=1, arrow_size=2):
    if type(data) is dict:  # layers
        for Cr, layer in data.items():
            xs, ys, ms = layer
            colors = np.array([manip_color(m) for m in ms])
            ax.scatter(xs, ys, c=colors, s=dot_size)
        dkeys = [int(k) for k in data.keys()]
    elif data.shape[1] == 4:  # raw
        raw = np.transpose(data)
        Crs, xs, ys, ms = raw
        colors = np.array([manip_color(m) for m in ms])
        ax.scatter(xs, ys, c=colors, s=dot_size)
        dkeys = [int(k) for k in set(Crs)]
    else:
        raise TypeError()
    ax.set_title('%s $C_{r}(deg)$:\n%s'% ('Feasibility' if is_feasibility else 'Manipulability', dkeys))
    ax.set_xlabel('x(cm)', fontsize=15)
    ax.set_ylabel('y(cm)', fontsize=15)
    ax.axis(xyMinMax)             
    ax.grid(True)
    
    # robot and object
    if is_feasibility:  # object
        marker_x = [1.0, -0.5, -0.5, -0.5]
        marker_y = [0.0, 0.0, -0.5, 0.5]
    else:  # robot
        marker_x = [1.0, -0.5, -0.5, 1.0]
        marker_y = [0.0, 0.5, -0.5, 0.0]
    arrow_x = np.array(marker_x) * arrow_size
    arrow_y = np.array(marker_y) * arrow_size
    ax.plot(arrow_x, arrow_y)

# fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2)
# xyminmax = [-20, 20, -20, 20]
# scatter_2d(ax1, manip_layers, xyminmax, is_feasibility=False)
# scatter_2d(ax2, self.feasi_raw, xyminmax)
# fig.tight_layout()
# plt.show()
    
def scatter_3d(ax, data, xyMinMax, is_feasibility=True, dot_size=2, arrow_size=2):
    if type(data) is dict:  # layers
        dkeys = [int(k) for k in data.keys()]
        raw = layers_to_raw(data)
    elif data.shape[1] == 4:  # raw
        dkeys = [int(k) for k in set(data[:,0])]
        raw = data
    else:
        raise TypeError()
    Crs, xs, ys, ms = np.transpose(raw)
    colors = np.array([manip_color(m) for m in ms])
    ax.scatter(xs, ys, zs=Crs, c=colors, s=dot_size)
    ax.set_xlabel('x(cm)', fontsize=15)
    ax.set_ylabel('y(cm)', fontsize=15)
    ax.set_zlabel('$C_{r}$(deg)', fontsize=15)
    ax.axis(xyMinMax)
    ax.set_title('Feasibility' if is_feasibility else 'Manipulability')
    
    # robot and object
    if is_feasibility:  # object
        marker_x = [1.0, -0.5, -0.5, -0.5]
        marker_y = [0.0, 0.0, -0.5, 0.5]
    else:  # robot
        marker_x = [1.0, -0.5, -0.5, 1.0]
        marker_y = [0.0, 0.5, -0.5, 0.0]
    arrow_x = np.array(marker_x) * arrow_size
    arrow_y = np.array(marker_y) * arrow_size
    for c in dkeys:
        arrow_z = np.array([c, c, c, c])
        ax.plot(arrow_x, arrow_y, arrow_z)

# fig = plt.figure()
# ax1 = fig.add_subplot(121, projection='3d')
# ax2 = fig.add_subplot(122, projection='3d')
# xyminmax = [-20, 20, -20, 20]
# scatter_3d(ax1, manip_layers, xyminmax, is_feasibility=False)
# scatter_3d(ax2, self.feasi_raw, xyminmax)
# plt.show()

### 1.1. Configuration

1. `Rsize`: the size of the robot-base
2. `M`: X-Y-Cr manipulability map

```
Cr Cr ...
x x ...
y y ... 
m m ...
```

In [2]:
Rsize = 5

In [3]:
M = np.load('fake_data.npy')
M = recarray_to_ndarray(M)
print(M.shape)
print(M)

(1575, 4)
[[-9.0000000e+01  0.0000000e+00  0.0000000e+00  1.2918387e-04]
 [-9.0000000e+01  0.0000000e+00  1.0000000e+00  0.0000000e+00]
 [-9.0000000e+01  0.0000000e+00  2.0000000e+00  0.0000000e+00]
 ...
 [ 9.0000000e+01  1.4000000e+01  1.2000000e+01  7.5058979e-01]
 [ 9.0000000e+01  1.4000000e+01  1.3000000e+01  5.8899295e-01]
 [ 9.0000000e+01  1.4000000e+01  1.4000000e+01  3.6736864e-01]]


### 1.2. Preparation

1. Convert the `M` to a feasibility map `F` (helical shape).

Cr, x, y, manipulability

In [4]:
class CollisionBox:
    def __init__(self, width, length, center, rad):
        """
                       | y, length
                       | 
           ------------- x, width
        """
        self.half_width = width / 2.0
        self.half_length = length / 2.0
        self.center = np.array(center)
        self.angle = rad
        self.minx, self.maxx, self.miny, self.maxy = (None, None, None, None)
        self.vertices = self._get_vertices()
        self.offsets = None
        self.offset_w = None
        self.offset_l = None
        self.inv_R = self._calc_rotation(-self.angle)
        
    @staticmethod
    def _calc_rotation(rad):
        c, s = np.cos(rad), np.sin(rad)
        return np.array(((c, -s),
                         (s,  c)))
    
    def _get_vertices(self):
        xy = np.array([[-self.half_width, -self.half_width,  self.half_width, self.half_width],
                       [ self.half_length, -self.half_length, -self.half_length, self.half_length]])
        R = self._calc_rotation(self.angle)
        vertices = np.matmul(R, xy)
        vertices[0] += self.center[0]
        vertices[1] += self.center[1]
        return vertices
        
    def set_offset(self, robot_radius):
        self.offset_w = w = self.half_width + robot_radius
        self.offset_l = l = self.half_length + robot_radius
        xy = np.array([[-w, -w,  w, w],
                       [ l, -l, -l, l]])
        R = self._calc_rotation(self.angle)
        offsets = np.matmul(R, xy)
        offsets[0] += self.center[0]
        offsets[1] += self.center[1]
        self.offsets = offsets.copy()
        self.minx = min(offsets[0])
        self.maxx = max(offsets[0])
        self.miny = min(offsets[1])
        self.maxy = max(offsets[1])
        
    def check(self, xy):
        if self._pseudo_check(xy):
            return self._detail_check(xy)
        return False
        
    def _pseudo_check(self, xy):
        x, y = xy
        if (self.minx <= x) and (x <= self.maxx):
            return (self.miny <= y) and (y <= self.maxy)
        return False
    
    def _detail_check(self, xy):
        local_xy = xy - self.center
        local_xy = np.matmul(self.inv_R, local_xy)
        x, y = local_xy
        return (abs(x) <= self.offset_w) and (abs(y) <= self.offset_l)
        
class FeasibilityMap:
    def __init__(self, manipulability_raw, robot_radius):
        self.robot_radius = robot_radius
        manip_layers = raw_to_layers(manipulability_raw)
        feasi_layers = self.manipulability_to_feasibility(manip_layers)
        self.feasi_raw = layers_to_raw(feasi_layers)
        
        self.two_plot([(manip_layers, False), (self.feasi_raw, True)])
        self.two_plot([(manip_layers, False), (self.feasi_raw, True)], dim='3d')
        
        """
        free_raw
        [Ct Cr x y m]
        """
        self.free_raw = None
        
        print(manip_layers[0].shape)
        print(feasi_layers[0].shape)
        print(self.feasi_raw.shape)
    
    @staticmethod
    def manipulability_to_feasibility(manipulability_layers):
        """
        manipulability_layers & feasibility_layers
            key: degree
            value: [(x, y, manip), ...]

        [rot]      | [Cr0]
        c0 -s0  0  | x1 x2 x3 x4 x5 ... x100
        s0  c0  0  | y1 y2 y3 y4 y5 ... y100
         0   0  1  | u1 u2 u3 u4 u5 ... u100
        """
        feasibility_layers = {}
        for Cr, mlayer in manipulability_layers.items():
            R = rotation_z(np.radians(180 - Cr))
            flayer = np.dot(R, mlayer)
            feasibility_layers[Cr] = flayer
        return feasibility_layers
    
    @staticmethod
    def two_plot(draws, dim='2d'):
        """
        draws: [(data, is_feasibility), (data, is_feasibility)]
        dim: 2d or 3d
        """
        xyMinMax = [-20, 20, -20, 20]
        idx = 0
        if dim == '2d':
            fig, axs = plt.subplots(nrows=1, ncols=2)
            for data, isF in draws:
                scatter_2d(axs[idx], data, xyMinMax, is_feasibility=isF)
                idx += 1
            fig.tight_layout()
        else:
            fig = plt.figure()
            axs = [fig.add_subplot(121, projection='3d'), fig.add_subplot(122, projection='3d')]
            for data, isF in draws:
                scatter_3d(axs[idx], data, xyMinMax, is_feasibility=isF)
                idx += 1
        plt.show()
        return plt
    
    def calc(self, Pt, Obs, Cr, Ct):
        ###################################
        print("Cut the range of `Cr` from `Fraw` and set it to `Fcut`.")
        minCr, maxCr = Cr
        print("min Cr: ", minCr)
        print("max Cr: ", maxCr)
        filter_arr = (self.feasi_raw[:,0] >= minCr) * (self.feasi_raw[:,0] <= maxCr)
        Fcut = self.feasi_raw[filter_arr].copy()
        
        # PLOT
        self.two_plot([(self.feasi_raw, True), (Fcut, True)])

        ###################################
        print("And extract only the maximum as a `Fmax`.")
        interval = 1  # cm
        sections = np.array(range(5, 20, interval))
        sq_sections = np.square(sections)
        print("sq_sections.shape: ", sq_sections.shape)
        
        num_sections = len(sq_sections) - 1
        Fmax = np.zeros((num_sections, 4), dtype=np.float32)        
        
        def find_idx(ascending, value):
            # Check range
            if ascending[0] > value or ascending[-1] < value:
                return None
            # Binary search
            Li = 0
            Ri = len(ascending) - 1
            while (Li + 1) < Ri:
                mid = int((Li + Ri) / 2)
                if value < ascending[mid]:
                    Ri = mid
                else:
                    Li = mid
            return Li
        
        for point in Fcut:
            x, y = point[1:3]
            sq_radius = x*x + y*y           
            idx = find_idx(sq_sections, sq_radius)
            if idx is not None:
                prev_manip = Fmax[idx, 3]
                new_manip = point[3]
                if prev_manip < new_manip:
                    Fmax[idx] = point.copy()
        print("Fmax.shape: ", Fmax.shape)
    
        # PLOT
        print("When Ct == 0.0 deg")
        fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2)
        scatter_2d(ax1, Fcut, [-20, 20, -20, 20])
        scatter_2d(ax2, Fmax, [-20, 20, -20, 20])
        for r in sections:
            ax1.add_artist(plt.Circle((0,0), radius=r, color='gray', alpha=0.4, fill=False))
            ax2.add_artist(plt.Circle((0,0), radius=r, color='gray', alpha=0.4, fill=False))
        fig.tight_layout()
        plt.show()
        
        ###################################
        print("Wipe the `Fmax` in the range of `Ct`. => `Fwiped`")
        minCt, maxCt = Ct
        print("min Ct: ", minCt)
        print("max Ct: ", maxCt)
        
        """
        Fwiped
        [Ct Cr x y m]
        """
        circle_points = []
        interval = 1  # cm
        for point in Fmax:
            Cr, x, y, m = point
            radius = np.sqrt(x*x + y*y)
            d_rad = interval / radius
            
            two_pi = 2.0 * np.pi
            num_points = int(two_pi / d_rad)
            half = int(num_points / 2)
            
            # Mapping to -180 ~ 180
            for idx in range(-half, num_points-half):
                rad = d_rad * idx
                Ct = np.degrees(rad)
                
                c, s = np.cos(rad), np.sin(rad)
                nx = c*x - s*y
                ny = s*x + c*y
                circle_points.append([Ct, Cr, nx, ny, m])
        Fwiped = np.array(circle_points)
        print(Fwiped.shape)
        print(Fwiped[:5])
        
        # PLOT
        wiped_plot = Fwiped[:,1:]
        fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2)
        scatter_2d(ax1, Fmax, [-20, 20, -20, 20])
        scatter_2d(ax2, wiped_plot, [-20, 20, -20, 20])
        for p in Fmax:
            cr, x, y, m = p
            r = np.sqrt(x*x + y*y)
            ax1.add_artist(plt.Circle((0,0), radius=r, color=manip_color(m), fill=False))
            ax1.add_artist(plt.Circle((x,y), radius=0.2, color="red", fill=False))
            ax2.add_artist(plt.Circle((x,y), radius=0.2, color="red", fill=False))
        fig.tight_layout()
        plt.show()
        
        ###################################
        print("Remove all obstacle areas from `Fwiped` with the offset of `Rsize`. => `Fclean`")
        """
        Fwiped
        [Ct Cr x y m]
        """
        # Pt, Obs
        target_center = np.array(Pt)
        Fclean = Fwiped.copy()
        print("Fclean.shape: ", Fclean.shape)
        for collision in Obs:
            collision.set_offset(self.robot_radius)
            filter_arr = [not collision.check(p[2:4] + target_center) for p in Fclean]
            Fclean = Fclean[filter_arr]
            print("Fclean.shape: ", Fclean.shape)
        
        self.free_raw = Fclean.copy()

        # PLOT
        clean_plot = Fclean[:,1:]
        fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2)
        scatter_2d(ax1, wiped_plot, [-20, 20, -20, 20])
        scatter_2d(ax2, clean_plot, [-20, 20, -20, 20])
        for p in Fmax:
            cr, x, y, m = p
            ax1.add_artist(plt.Circle((x,y), radius=0.2, color="red", fill=False))
            ax2.add_artist(plt.Circle((x,y), radius=0.2, color="red", fill=False))
        for collision in Obs:
            xs, ys = collision.vertices
            xs = np.append(xs, xs[0])
            ys = np.append(ys, ys[0])
            ax1.plot(xs, ys)
            ax2.plot(xs, ys)
            xs, ys = collision.offsets
            xs = np.append(xs, xs[0])
            ys = np.append(ys, ys[0])
            ax1.plot(xs, ys)
            ax2.plot(xs, ys)
        fig.tight_layout()
        plt.show()
    
    def get_candidates(self, num=1):
        """
        self.free_raw (descending order)
        [Ct Cr x y m]
        """
        # Sorting in descending order
        ms = -np.transpose(self.free_raw)[4]
        s = ms.argsort()
        Fsort = self.free_raw[s]
        
        print("Fclean[:10]: ",self.free_raw[:10])
        print("Fclean[:10]: ",Fsort[:10])
        
        candidates = Fsort[:10].copy()
        
        # PLOT
        clean_plot = self.free_raw[:,1:]
        fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2)
        scatter_2d(ax1, clean_plot, [-20, 20, -20, 20])
#         scatter_2d(ax2, clean_plot, [-20, 20, -20, 20])
        for p in candidates:
            ct, cr, x, y, m = p
            ax1.add_artist(plt.Circle((x,y), radius=0.2, color="red", fill=False))
        fig.tight_layout()
        plt.show()
        
        return candidates

In [5]:
a = np.array([1, 2, 3, 4, 5, 6])
(a > 3) * (a< 6)

array([False, False, False,  True,  True, False])

In [6]:
F = FeasibilityMap(M, Rsize)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(3, 225)
(3, 225)
(1575, 4)


## 2. Online

### 2.1. Input

Known
1. `self.feasi_layers`: feasibility map
2. `self.robot_radius`: robot base (circle radius)

---
Input

1. `Pt`: Position of the target object (relative to the robot’s current pose) => `(x, y)`
2. `Obs`: Area list of ground obstacles => `[CollisionModel, ...]`
3. `Cr`: Constraints on the approach angle (relative to the robot heading) (-90 ~ 90) => `(min, max)`
4. `Ct`: Constraints on the approach angle (relative to the target heading) (-180 ~ 180) => `(min, max)`

In [7]:
Pt = (0.0, 0.0)
Obs = [CollisionBox(30.0, 15.0, (5.0, 5.0), np.radians(-20.0))]
Cr = (30, 30)  # min, max
Ct = (-180, 180) # min, max

### 2.2. Process

1. Cut the range of `Cr` from `F` and set it to `Fcut`.
2. ~~[SKIP] Scan the maximum points for the radius and angle by each circle in `Fcut`.~~
3. And extract only the maximum as a `Fmax`.
4. Wipe the `Fmax` in the range of `Ct`.
5. Remove all obstacle areas from `Fwiped` with the offset of `Rsize`.

In [8]:
F.calc(Pt, Obs, Cr, Ct)

Cut the range of `Cr` from `Fraw` and set it to `Fcut`.
min Cr:  30
max Cr:  30


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

And extract only the maximum as a `Fmax`.
sq_sections.shape:  (15,)
Fmax.shape:  (14, 4)
When Ct == 0.0 deg


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Wipe the `Fmax` in the range of `Ct`. => `Fwiped`
min Ct:  -180
max Ct:  180
(1029, 5)
[[-172.1854839    30.            5.2142901     2.1934404     0.86988288]
 [-162.05692602   30.            4.74729603    3.07622861    0.86988288]
 [-151.92836815   30.            4.13233492    3.86313477    0.86988288]
 [-141.79981027   30.            3.38857429    4.52963203    0.86988288]
 [-131.67125239   30.            2.53919621    5.05494656    0.86988288]]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Remove all obstacle areas from `Fwiped` with the offset of `Rsize`. => `Fclean`
Fclean.shape:  (1029, 5)
Fclean.shape:  (348, 5)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### 2.3. Output

1. Candidate poses (sorted in descending order of manipulability)

In [9]:
candidates = F.get_candidates()

Fclean[:10]:  [[44.74048647 30.         -3.81767566 -5.14055982  0.86988288]
 [53.68858376 30.         -2.97165216 -5.67179744  0.86988288]
 [62.63668106 30.         -2.05329654 -6.06497948  0.86988288]
 [71.58477835 30.         -1.08496222 -6.31053562  0.86988288]
 [32.4113852  30.         -4.7852017  -5.20594349  0.95477819]
 [40.51423151 30.         -4.0036495  -5.82844693  0.95477819]
 [48.61707781 30.         -3.14215769 -6.3345756   0.95477819]
 [56.71992411 30.         -2.21792741 -6.71422378  0.95477819]
 [64.82277041 30.         -1.24941246 -6.95981117  0.95477819]
 [72.92561671 30.         -0.25595088 -7.06643418  0.95477819]]
Fclean[:10]:  [[ 1.87678554e+01  3.00000000e+01 -1.43092758e+01 -5.31456762e+00
   1.00000000e+00]
 [ 6.85323890e+01  3.00000000e+01 -2.99820501e+00 -1.38928315e+01
   1.00000000e+00]
 [ 7.25637060e+01  3.00000000e+01 -2.01409687e+00 -1.40692364e+01
   1.00000000e+00]
 [ 7.65950230e+01  3.00000000e+01 -1.02002206e+00 -1.41760204e+01
   1.00000000e+00]
 

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …